# Introduction and TCGA background

Liver cancer has the second highest worldwide cancer mortality rate and has limited therapeutic options.

The most common form of liver cancer is hepatocellular carcinoma (HCC), making up 80% of liver cancer cases in the US. HCC is the 9th leading cause of cancer-related deaths in the US, and third worldwide.[1](https://www.cancer.gov/about-nci/organization/ccg/research/structural-genomics/tcga/studied-cancers/liver)


A consortium known as The Cancer Genome Atlas (TCGA) network has made a plethora of molecular information on various cancers, including HCC publicly available. This data include genetic, molecular, diagnostic, demographic, and clinical information from over 350 patients with HCC. 

While surgery can be an effective method for treating early-stage liver cancer, there are few options for more advanced and metastatic tumors. Sorafenib and regorafinib are the only approved treatments for managing advanced HCC, and many others potential therapeutics have failed in Phase III clinical trials. [Previous studies](https://www.cell.com/cell/fulltext/S0092-8674(17)30639-6) studying genomic alterations in samples have found frequently-mutated genes LZTR1, EEF1A1, SF3B1, and SMARCA4.

In addition to DNA alterations however, different biological and tumor microenvrionment factors can influence disease progression. A transcriptomic survey of tissues at various stages of disease progression could help elucidate some of the underlying pathways contributing to tumorigenesis.

## RNA Seq

In this post, we'll be exploring a liver cancer RNA-seq dataset from TCGA. TCGA represents one of the most comprehensive publically-available datasets for cancer research. This tutorial will walk through several steps in how to obtain, process, explore and model gene expression data as it relates to cancer genomics.


In biology, copies of our DNA are used as a template for creating different biological products and proterins. This intermediate copy is known as RNA. Technological advances have provided new ways to access and read RNA from tissues and even single cells. The use of sequencing technology to read RNA is known as RNA-seq.

RNA-seq can provide a granular way to understand which genes are being actively expressed (or suppressed) at any given time, and provide insight into the role of certain gentic components in influencing tumor progression.